# MWPCCC Data Structuring & Automation for Donor Tracking & Fundraising

**Author:** Geneva Burleigh  
**Date:** 10/15/2024 - 11/20/2024
**Project:** Automating Donor Tracking & Fundraising Analysis with Google Sheets  

---

## 1. Project Overview

### 1.1 Objective
This project was initiated to solve a long-standing gap in donor tracking and fundraising analysis at Mt Washington Preschool and Child Care Center (MWPCCC), a 501(c)3 non-profit childcare organization in Los Angeles. Despite over 30 years of operations, donor information was scattered across spreadsheets, accounting software, and in situ operational software. Without a centralized system, outreach efforts were inconsistent, and data-driven decision-making was impossible.

To address this, I structured and cleaned donor data in Google Sheets, implemented unique donor IDs, automated reporting with Google Apps Script, and prepared the dataset for migration to PostgreSQL for deeper analysis.

### 1.2 Tools & Technologies Used

| Tool | Purpose |
|------|---------|
| **Google Sheets** | Main data repository before migration |
| **Google Apps Script** | Automating ID generation, data aggregation, and anonymization |
| **Excel** | Data formatting, duplicate removal, and validation |
| **Regular Expressions (RegEx)** | Cleaning and standardizing text data |
| **Data Validation & Dropdowns** | Standardizing entries to minimize errors |

---

## 2. Initial Data Challenges

### 2.1 Challenges in Data Consolidation  

Before structuring this data, donor and fundraising records were **not stored in a centralized location**. Instead, they were scattered across:  

- **Procare (Childcare Management System)** – Contained **some** parent contact data but was inconsistently formatted across reports.  
- **QuickBooks (Accounting Software)** – Held **financial transaction records** but had no direct links to donor names or engagement history.  
- **Mailchimp (Email Marketing Platform)** – Provided **limited engagement history**, but only for donors who had been sent email campaigns.  
- **Personal Contact Lists** – Some donor information was recovered through personal knowledge and outreach.  

**Rebuilding the Dataset:**  
Because of these limitations, the process required:  
✅ **Manually linking contact details** across platforms to ensure donors were correctly identified.  
✅ **Reconstructing donation histories** where records were missing or incomplete.  
✅ **Using judgment calls** to group donations under `GivingEntityID` when no direct match existed.  

This project was not just about cleaning up a messy dataset — it was about **forensic data reconstruction** from scattered sources. The final structured version in Google Sheets allowed for **accurate tracking, automation, and ultimately a successful migration into PostgreSQL for analysis.**

### 2.2 Goals for Data Structuring
✅ **Standardize formatting** for all donor details.  
✅ **Implement unique GivingEntityID** to track donations by individuals, families, or corporate entities.  
✅ **Automate donation tracking & aggregation** to remove manual work.  
✅ **Ensure data integrity** before migrating to a PostgreSQL database.  

---

## 3. Data Structuring Process

### 3.1 Sheets & Column Standardization
**Key Sheets Created:**
1. **Master Donor List** – Unique GivingEntityID, contact details, total donations.
2. **Individual Donors** – List of individuals tied to their respective GivingEntityID.
3. **Corporate Donors** – Similar to Individual Donors but for businesses or foundations.
4. **Donations** – Each donation entry, linked to an individual and a GivingEntityID.
5. **Fundraisers** – Tracks the success of each fundraising event.
6. **Fundraising Costs** – Tracks the cost of each fundraising event.
7. **Summary Sheet** – Aggregated donor data for reporting.
8. **Processed Data** – Prepared dataset for generating mailings and reports.

### 3.2 Creating Unique Identifiers (GivingEntityID)
Implemented a **Google Apps Script** to generate a **GivingEntityID** that:
- Groups individuals by shared address and family ties.
- Ensures that donations by a family are aggregated properly.
- Distinguishes between individuals, families, and corporate entities.

➡ **[View Full Script Here](/work/Scripts/AssignGivingEntityIDs.js)**

```javascript
function assignUniqueGivingEntityIDs() {
  var sheet = SpreadsheetApp.getActiveSpreadsheet().getSheetByName("Master Donor List");
  var data = sheet.getDataRange().getValues();
  var idCounter = 1000;
  
  for (var i = 1; i < data.length; i++) {
    if (!data[i][0]) { // Column A = GivingEntityID
      data[i][0] = "GE" + (idCounter++);
    }
  }
  sheet.getDataRange().setValues(data);
}
```

---

## 4. Automation with Google Apps Script

### 4.1 Matching Donations with Donors
This script ensures that each donation is correctly linked to a `GivingEntityID`.

➡ **[View Full Script Here](/work/Scripts/populateDonationGivingEntityIDs.js)**

```javascript
function populateDonationGivingEntityIDs() {
  var ss = SpreadsheetApp.getActiveSpreadsheet();
  var donationsSheet = ss.getSheetByName("Donations");
  var masterSheet = ss.getSheetByName("Master Donor List");

  var donationsData = donationsSheet.getDataRange().getValues();
  var masterData = masterSheet.getDataRange().getValues();

  var donorMap = new Map();
  masterData.slice(1).forEach(row => {
    donorMap.set(row[2], row[0]); // Match Primary Contact to GivingEntityID
  });

  for (var i = 1; i < donationsData.length; i++) {
    var donorName = donationsData[i][2];
    if (donorMap.has(donorName)) {
      donationsData[i][0] = donorMap.get(donorName);
    }
  }
  donationsSheet.getDataRange().setValues(donationsData);
}
```

### 4.2 Other Utility Scripts
These scripts perform smaller tasks that support the core automation:

- **[Populate GivingEntityIDs](/work/Scripts/PopulateGivingEntityIDs.js)** – Ensures all donor sheets are linked to the `GivingEntityID`.  
- **[Generate Unique Donation IDs](/work/Scripts/populateDonationIDs.js)** – Assigns unique IDs to donations for tracking.  
- **[Log Fundraiser Costs](/work/Scripts/updateFundraiserCosts.js)** – Stores costs associated with each fundraiser.  
- **[Calculate Profit from Fundraisers](/work/Scripts/updateFundraiserProfits.js)** – Calculates `Total Raised - Costs`.

---

## 5. Anonymization for Public Sharing
Since donor data contains **sensitive information**, a script was written to replace real names, addresses, emails, and phone numbers with **randomized but structured placeholders.**

➡ **[View Full Script Here](/work/Scripts/anonymizeData.js)**

---

## 6. Impact & Results
### 6.1 Improvements Achieved
| Issue | Before | After |
|-------|--------|------|
| Data Consistency | Inconsistent names & formats | Standardized formatting via scripts |
| Tracking Donations | Manual entry | Automated using IDs & formulas |
| Duplicate Donors | Same donor appearing multiple times | Unique `GivingEntityID` implemented |
| Mail Merge Prep | Manual formatting | Auto-generated mailing lists |

### 6.2 Key Takeaways
- **Google Sheets is powerful** but requires structure for effective data management.
- **Automation reduces errors** and saves hours of manual work.
- **Unique Identifiers are critical** for linking donations, fundraisers, and donors.
- **Anonymization enables data sharing** without privacy concerns.

---

## 7. Next Steps & Migration to PostgreSQL
With the cleaned and structured dataset, the next step was **migrating this data into PostgreSQL** for:
✅ **Advanced querying & reporting** using SQL.  
✅ **Improved data integrity** with relational database constraints.  
✅ **Faster insights** via optimized queries.  

📌 **See the PostgreSQL migration & reporting project here:** [Data Migration and Analysis Report](https://deepnote.com/workspace/puddles-7d469830-b020-4998-9332-fad683944541/project/Welcome-d50b48dc-8b60-4e72-885a-59c1190a91a3/notebook/Data-Migration-and-Analysis-Report-870451d5ae6d45a5bca1bd2f825144ae?utm_content=d50b48dc-8b60-4e72-885a-59c1190a91a3)

